# Evidently Data Drift Metrics

In [ ]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_openml

## Prepare Data

In [ ]:
# load "adult" dataset
data = fetch_openml(name='adult', version=2, as_frame='auto')
df = data.frame

# prepare target and prediction
df['target'] = df['education-num']
df['prediction'] = df['education-num'].values + np.random.normal(0, 6, df.shape[0])

# make reference and current datasets
reference_data = df[~df.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
current_data = df[df.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
current_data.head()

## One column data drift

In [ ]:
from evidently.metrics import ColumnDriftMetric
from evidently.report import Report

report = Report(metrics=[ColumnDriftMetric(column_name="class")])
report.run(current_data=current_data, reference_data=reference_data)
report

In [ ]:
report.json()

In [ ]:
report = Report(metrics=[ColumnDriftMetric(column_name="target")])
report.run(current_data=current_data, reference_data=reference_data)
report

In [ ]:
report.json()

## Data Drift for all columns

In [ ]:
from evidently.metrics import DataDriftTable


report = Report(metrics=[DataDriftTable()])
report.run(current_data=current_data, reference_data=reference_data)
report

In [ ]:
report.json()

## Owerall dataset drift

In [ ]:
from evidently.metrics import DatasetDriftMetric

report = Report(metrics=[DatasetDriftMetric()])
report.run(current_data=current_data, reference_data=reference_data)
report

In [ ]:
report = Report(metrics=[DatasetDriftMetric(drift_share=0.4)])
report.run(current_data=current_data, reference_data=reference_data)
report

## One Column Value Plot Metric

In [ ]:
from evidently.metrics import ColumnValuePlot

report = Report(metrics=[ColumnValuePlot(column_name="age")])
report.run(current_data=current_data, reference_data=reference_data)
report

## Data Drift tests

In [ ]:
from evidently.test_preset import NoTargetPerformanceTestPreset
from evidently.test_suite import TestSuite


data_quality_suite = TestSuite(
    tests=[
        NoTargetPerformanceTestPreset(columns=["education-num", "hours-per-week"], stattest="psi"),
    ]
)

data_quality_suite.run(current_data=current_data, reference_data=reference_data)
data_quality_suite